# Pre requisites

Install and import required packages

In [1]:
!pip install boto3
!pip install tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 1.9 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')
paths = '/content/drive/MyDrive/Pytorch_Projects/Project_1'

Mounted at /content/drive


In [3]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from tqdm import tqdm
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [4]:
data_df = pd.read_csv(paths + '/data/riceClassification.csv')
data_df.head()

,id,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,EquivDiameter,Extent,Perimeter,Roundness,AspectRation,Class
0,1,4537,92.229316,64.012769,0.719916,4677,76.004525,0.657536,273.085,0.764510,1.440796,1
1,2,2872,74.691881,51.400454,0.725553,3015,60.471018,0.713009,208.317,0.831658,1.453137,1
2,3,3048,76.293164,52.043491,0.731211,3132,62.296341,0.759153,210.012,0.868434,1.465950,1
3,4,3073,77.033628,51.928487,0.738639,3157,62.551300,0.783529,210.657,0.870203,1.483456,1
4,5,3693,85.124785,56.374021,0.749282,3802,68.571668,0.769375,230.332,0.874743,1.510000,1


In [5]:
data_df.dropna(inplace=True)
data_df.drop(['id'],axis=1, inplace=True)
print(data_df.shape)
print(data_df['Class'].unique())
print(data_df['Class'].value_counts())

(18185, 11)
[1 0]
Class
1    9985
0    8200
Name: count, dtype: int64


In [6]:
original_df = data_df.copy()
for column in data_df.columns:
  data_df[column] = data_df[column]/data_df[column].abs().max()

In [7]:
X=np.array(data_df.iloc[:,:-1])
y=np.array(data_df.iloc[:,-1])

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test,test_size=0.2)
print(X_train.shape)
print(X_test.shape)
print(X_val.shape)


(14548, 10)
(2909, 10)
(728, 10)


In [9]:
class dataset(Dataset):
  def __init__(self,X,y):
      self.X =torch.tensor(X,dtype=torch.float32).to(device)
      self.y =torch.tensor(y,dtype=torch.long).to(device)
  def __len__(self):
    return len(self.X)
  def __getitem__(self,index):
    return self.X[index] ,self.y[index]


In [10]:
training_data = dataset(X_train,y_train)
validation_data = dataset(X_val,y_val)
testing_data = dataset(X_test,y_test)

In [11]:
train_dataloader = DataLoader(training_data, batch_size=8, shuffle=True)
val_dataloader = DataLoader(validation_data,batch_size=8,shuffle=True)
test_dataloader = DataLoader(testing_data,batch_size=8,shuffle=True)